# Deploying PaddleOCR in SageMaker Studio

(Optional) Update SageMaker to the latest version

In [ ]:
# Optional to run the below

import sys

!{sys.executable} -m pip install -q --upgrade sagemaker

## Overview

The **SageMaker Python SDK** helps you deploy your models for training and hosting in optimized, productions ready containers in SageMaker. The SageMaker Python SDK is easy to use, modular, extensible and compatible with TensorFlow, MXNet, PyTorch and Chainer. This tutorial focuses on how to deploy a PaddleOCR model to capture the text values of captured license plate images.

### Set up the environment

Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data. This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these. Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the sagemaker.get_execution_role() with appropriate full IAM role arn string(s).

In [ ]:
import boto3
import json
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role
from sagemaker.utils import name_from_base
from sagemaker.session import Session

session = Session()
region = session.boto_region_name
bucket = session.default_bucket()
role = get_execution_role()

## Compose Dockerfile for the base image used for deployment

In [ ]:
%%writefile Dockerfile
FROM registry.baidubce.com/paddlepaddle/paddle:2.1.0 as build
# To use GPU mode, use below base image instead
# FROM nvidia/cuda:10.2-cudnn7-devel-ubuntu16.04
    
# Set a docker label to advertise multi-model support on the container
LABEL com.amazonaws.sagemaker.capabilities.multi-models=true
# Set a docker label to enable container to use SAGEMAKER_BIND_TO_PORT environment variable if present
LABEL com.amazonaws.sagemaker.capabilities.accept-bind-to-port=true

ARG PYTHON_VERSION=3.7.10
ARG MMS_VERSION=1.0.8

# See http://bugs.python.org/issue19846
ENV LANG C.UTF-8
ENV LD_LIBRARY_PATH /opt/conda/lib/:$LD_LIBRARY_PATH
ENV PATH /opt/conda/bin:$PATH


RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    ca-certificates \
    cmake \
    curl \
    git \
    jq \
    libgl1-mesa-glx \
    libglib2.0-0 \
    libsm6 \
    libxext6 \
    libxrender-dev \
    openjdk-8-jdk-headless \
    vim \
    wget \
    zlib1g-dev

RUN curl -L -o ~/miniconda.sh https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh \
 && chmod +x ~/miniconda.sh \
 && ~/miniconda.sh -b -p /opt/conda \
 && rm ~/miniconda.sh \
 && /opt/conda/bin/conda update conda \
 && /opt/conda/bin/conda install -y \
    python=$PYTHON_VERSION \
    cython==0.29.12 \
    ipython==7.7.0 \
    mkl-include==2019.4 \
    mkl==2019.4 \
    numpy==1.16.4 \
    scipy==1.3.0 \
    typing==3.6.4 \
 && /opt/conda/bin/conda clean -ya

# install paddleocr
RUN pip3 install paddlepaddle # Change to "RUN pip3 install paddlepaddle-gpu==2.0.2" for using GPU
RUN pip3 install "paddleocr>=2.0.1" 

# Install MXNet, MMS, and SageMaker Inference Toolkit to set up MMS
RUN pip3 --no-cache-dir install mxnet \
                                multi-model-server \
                                sagemaker-inference \
                                retrying

# Copy entrypoint script to the image
COPY dockerd-entrypoint.py /usr/local/bin/dockerd-entrypoint.py
RUN chmod +x /usr/local/bin/dockerd-entrypoint.py

RUN mkdir -p /home/model-server/

# Copy the default custom service file to handle incoming data and inference requests
COPY model_handler.py /home/model-server/model_handler.py

# Define an entrypoint script for the docker image
ENTRYPOINT ["python", "/usr/local/bin/dockerd-entrypoint.py"]

# Define command to be passed to the entrypoint
CMD ["serve"]

## Compose .py files for inference entry point and model handling

In [ ]:
%%writefile dockerd-entrypoint.py
import subprocess
import sys
import shlex
import os
from retrying import retry
from subprocess import CalledProcessError
from sagemaker_inference import model_server

def _retry_if_error(exception):
    return isinstance(exception, CalledProcessError or OSError)

@retry(stop_max_delay=1000 * 50,
       retry_on_exception=_retry_if_error)
def _start_mms():
    # by default the number of workers per model is 1, but we can configure it through the
    # environment variable below if desired.
    # os.environ['SAGEMAKER_MODEL_SERVER_WORKERS'] = '2'
    model_server.start_model_server(handler_service='/home/model-server/model_handler.py:handle')

def main():
    if sys.argv[1] == 'serve':
        _start_mms()
    else:
        subprocess.check_call(shlex.split(' '.join(sys.argv[1:])))

    # prevent docker exit
    subprocess.call(['tail', '-f', '/dev/null'])
    
main()

In [ ]:
%%writefile model_handler.py
"""
ModelHandler defines an example model handler for load and inference requests for MXNet CPU models
"""
from collections import namedtuple
import json
import logging
import os
import re

import mxnet as mx
import numpy as np
from io import BytesIO
from paddleocr import PaddleOCR, draw_ocr
from glob import glob
from PIL import Image
import base64

class ModelHandler(object):
    """
    A sample Model handler implementation.
    """

    def __init__(self):
        self.initialized = False
        self.ocr = None

    def initialize(self, context):
        """
        Initialize model. This will be called during model loading time
        :param context: Initial context contains model server system properties.
        :return:
        """
        self.initialized = True
        model_dir = "/opt/ml/model/"
        
        # Load ocr model, change "use_gpu=False" to "use_gpu=True" in case of using GPU
        try:
            self.ocr = PaddleOCR(det_model_dir=os.path.join(model_dir,'model/det'),
            rec_model_dir=os.path.join(model_dir,'model/rec'),
            cls_model_dir=os.path.join(model_dir,'model/cls'), 
            use_angle_cls=True, lang="ch", use_gpu=False)
              
        except Exception as e:
            raise
            
    def preprocess(self, request):
        """
        Transform raw input into model input data.
        :param request: list of raw requests
        :return: list of preprocessed model input data
        """
        # Take the input data and pre-process it make it inference ready

        img_list = []
        for idx, data in enumerate(request):
            # Read the bytearray of the image from the input
            img_arr = data.get('body')
            img_arr = base64.b64decode(img_arr)
            img_arr = Image.open(BytesIO(img_arr))
            img_arr = np.array(img_arr)
            
            # Check the number of dimension
            assert len(img_arr.shape) == 3, "Dimension must be 3, but {}".format(len(img_arr.shape)) 

            img_list.append(img_arr)

        return img_list

    def inference(self, model_input):
        """
        Internal inference methods
        :param model_input: transformed model input data list
        :return: list of inference output 
        """
        res_list = []
        # Do some inference call to engine here and return output
        for img in model_input:
            result = self.ocr.ocr(img, cls=True)
            for res in result:
                ## because float32 is not json serializable, score is converted to float (float64)
                ## However the score is in tuple and cannot be replaced. The entire tupple is replaced as list.
                string = res[1][0]
                score = float(res[1][1])
                res[1] = [string,score]

            res_list.append(result)
        return res_list
        
    def handle(self, data, context):
        """
        Call preprocess, inference and post-process functions
        :param data: input data
        :param context: mms context
        """
        
        model_input = self.preprocess(data)
        model_out = self.inference(model_input)
        return model_out

_service = ModelHandler()


def handle(data, context):
    if not _service.initialized:
        _service.initialize(context)

    if data is None:
        return None

    return _service.handle(data, context)

## Install libraries for building container images in SageMaker Studio

In [ ]:
!pip install sagemaker-studio-image-build 

## Build container image

In [ ]:
!sm-docker build . --repository paddleocr-on-sagemaker-example:latest

## Print the uri of the container image built and pushed to Amazon ECR

In [ ]:
account_id = boto3.client('sts').get_caller_identity().get('Account')
region = boto3.session.Session().region_name 
image_uri = str(account_id) + ".dkr.ecr." +  region + ".amazonaws.com/paddleocr-on-sagemaker-example:latest"

print(image_uri)

## Download the inference models from [PaddleOCR](https://github.com/PaddlePaddle/PaddleOCR)

In [ ]:
!mkdir -p model/det
!mkdir -p model/rec
!mkdir -p model/cls

# Detection
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_det_infer.tar -O model/det/ch_ppocr_mobile_v2.0_det_infer.tar
!cd model/det/ && tar xvf ch_ppocr_mobile_v2.0_det_infer.tar --strip-components 1 && rm ch_ppocr_mobile_v2.0_det_infer.tar

# Recognition
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_rec_infer.tar -O model/rec/ch_ppocr_mobile_v2.0_rec_infer.tar
!cd model/rec/ && tar xvf ch_ppocr_mobile_v2.0_rec_infer.tar --strip-components 1 && rm ch_ppocr_mobile_v2.0_rec_infer.tar

# Directoin Classificatoin
!wget https://paddleocr.bj.bcebos.com/dygraph_v2.0/ch/ch_ppocr_mobile_v2.0_cls_infer.tar -O model/cls/ch_ppocr_mobile_v2.0_cls_infer.tar
!cd model/cls/ && tar xvf ch_ppocr_mobile_v2.0_cls_infer.tar --strip-components 1 && rm ch_ppocr_mobile_v2.0_cls_infer.tar

!tar -zcvf model.tar.gz model

## Upload the packed model artifact to Amazon S3

In [ ]:
import sagemaker 

model_uri = sagemaker.Session().upload_data("model.tar.gz", key_prefix="ocr_model")
model_uri

## Define model with inputs from the built container image and packed model artifact

In [ ]:
from sagemaker.model import Model
from sagemaker.predictor import Predictor

ocr_model = sagemaker.model.Model(image_uri,
                      model_data=model_uri, 
                      predictor_cls=Predictor,
                      role=sagemaker.get_execution_role())

## Deploy the model

In [ ]:
# You may change the instance type to a smaller or larger one based on your performance requirement and cost considerations
predictor = ocr_model.deploy(initial_instance_count=1,instance_type="ml.c5.4xlarge")

# Get the endpoint name of the deployed model
print(predictor.endpoint_name)

## Testing the deployed model

In [ ]:
from PIL import Image, ImageDraw, ImageFont
import base64
import io
from io import BytesIO
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

directory = ''
file = 'your-file-name'
file_path = directory + file 
with open(file_path, 'rb') as image:
    stream = io.BytesIO(image.read())
    image = Image.open(stream)

buffered = BytesIO()
image.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue())

import json
response = json.loads(predictor.predict(img_str))
response

In [ ]:
!conda install glib -y
!pip install albumentations

import numpy as np
import cv2

x_offset = 20
y_offset = 0
for i, res in enumerate(response):
    box = np.reshape(np.array(res[0]), [-1, 1, 2]).astype(np.int64)
    image = cv2.putText(np.array(image), '('+str(i)+')', (box[0][0][0] -x_offset, box[0][0][1]-y_offset), 
                        cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 1, cv2.LINE_AA)
    image = cv2.polylines(np.array(image), [box], True, (255, 0, 0), 2)

import matplotlib.pyplot as plt
for i,res in enumerate(response):
    print('('+str(i)+'): '+res[1][0], end=', ')
fig = plt.figure(dpi=200)
plt.imshow(image)
plt.show()

## Optional cleanup
When you're done with the endpoint, you should clean it up.

All of the training jobs, models and endpoints we created can be viewed through the SageMaker console of your AWS account.

In [ ]:
# predictor.delete_endpoint()